In [18]:
import boto3
import tempfile
import logging
import os
import base64

In [2]:
import logging
logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S')
logger = logging.getLogger()
logger.setLevel(logging.INFO)


In [3]:
logging.debug('hello')

In [4]:
# Let's use Amazon S3
s3 = boto3.resource('s3')

2021-11-04 12:37:04 INFO     Found credentials in environment variables.


In [5]:
# Tuplex needs a bucket.
# => create one tuplex-<iam-user> per default.
# => this is where stuff gets stored.

# layout bucket like this:
# tuplex-<iam-user>/notebooks
# tuplex-<iam-user>/data
# tuplex-<iam-user>/scratch

# upload lambda function as
# tuplex-runner

# -> add versioning to tuplex-runner! => allow for auto upload?


# Print out bucket names
for bucket in s3.buckets.all():
    print(bucket.name)

aws-deepracer-3f4fbafa-e09c-412c-8491-baeb4b0bffb7
bbsn00
bmwcpo
tuplex
tuplex-leonhard
tuplex-public
tuplex-test


In [6]:
def current_iam_user():
    iam = boto3.resource('iam')
    user = iam.CurrentUser()
    return user.user_name.lower()

def default_lambda_name():
    return 'tuplex-lambda-runner'

def default_lambda_role():
    return 'tuplex-lambda-role'

def default_bucket_name():
    return 'tuplex-' + current_iam_user()

def current_region():
    session = boto3.session.Session()
    region = session.region_name
    return region

def setup_aws(iam_user=current_iam_user(),
              lambda_name=default_lambda_name(),
              lambda_role=default_lambda_role(),
              region=current_region()
              ):
    logging.info('Setting up AWS Lambda backend for IAM user {}'.format(iam_user))
    logging.info('Configuring backend in zone: {}'.format(region))
    
    # check if iam user is found?
    # --> skip for now, later properly authenticate using assume_role as described in
    # https://docs.aws.amazon.com/IAM/latest/UserGuide/id_roles_use_switch-role-api.html
    
    # step 1: create Lambda role if not exists
    iam = boto3.resource('iam')
    
    
   
    
setup_aws()

2021-11-04 12:37:05 INFO     Setting up AWS Lambda backend for IAM user leonhard
2021-11-04 12:37:05 INFO     Configuring backend in zone: us-east-1


In [7]:
iam = boto3.resource('iam')
iam_client = boto3.client('iam')

In [8]:
lambda_role=default_lambda_role()

region = current_region()
overwrite = True


def create_lambda_role(iam_client, lambda_role):
    
    # Roles required for AWS Lambdas
    trust_policy = '{"Version":"2012-10-17","Statement":[{"Effect":"Allow","Principal":{"Service":"lambda.amazonaws.com"},"Action":"sts:AssumeRole"}]}'
    lambda_access_to_s3 = '{"Version":"2012-10-17","Statement":[{"Effect":"Allow","Action":["s3:*MultipartUpload*","s3:Get*","s3:ListBucket","s3:Put*"],"Resource":"*"}]}'
    lambda_invoke_others = '{"Version":"2012-10-17","Statement":[{"Effect":"Allow","Action":["lambda:InvokeFunction","lambda:InvokeAsync"],"Resource":"*"}]}'

    iam_client.create_role(RoleName=lambda_role,
                           AssumeRolePolicyDocument=trust_policy,
                           Description='Auto-created Role for Tuplex AWS Lambda runner')
    iam_client.attach_role_policy(RoleName=lambda_role, PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole')
    iam_client.put_role_policy(RoleName=lambda_role, PolicyName='InvokeOtherlambdas', PolicyDocument=lambda_invoke_others)
    iam_client.put_role_policy(RoleName=lambda_role, PolicyName='LambdaAccessForS3', PolicyDocument=lambda_access_to_s3)
    logging.info('Created Tuplex AWS Lambda runner role ({})'.format(lambda_role))
    
    # check it exists
    try:
        response = iam_client.get_role(RoleName=lambda_role)
        print(response)
    except:
        raise Exception('Failed to create AWS Lambda Role')
    
def remove_lambda_role(iam_client, lambda_role):
    
    # detach policies...
    try:
        iam_client.detach_role_policy(RoleName=lambda_role, PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole')
    except Exception as e:
        logging.error('Error while detaching policy AWSLambdaBasicExecutionRole, Tuplex setup corrupted? Details: {}'.format(e))
        
    policy_names = iam_client.list_role_policies(RoleName=lambda_role)['PolicyNames']
    
    for name in policy_names:
        try:
            iam_client.delete_role_policy(RoleName=lambda_role, PolicyName=name)
        except Exception as e:
            logging.error('Error while detaching policy {}, Tuplex setup corrupted? Details: {}'.format(name, e))
        
    # delete role...
    iam_client.delete_role(RoleName=lambda_role)

def setup_lambda_role(iam_client, lambda_role, region, overwrite):
    try:
        response = iam_client.get_role(RoleName=lambda_role)
        logging.info('Found Lambda role from {}'.format(response['Role']['CreateDate']))

         # throw dummy exception to force overwrite
        if overwrite:
            remove_lambda_role(iam_client, lambda_role)
            logging.info('Overwriting existing role {}'.format(lambda_role))
            create_lambda_role(iam_client, lambda_role)

    except iam_client.exceptions.NoSuchEntityException as e:
        logging.info('Role {} was not found in {}, creating ...'.format(lambda_role, region))
        create_lambda_role(iam_client, lambda_role)

## Creating/specifying s3 scratch space

In [9]:
s3_client = boto3.client('s3', region_name=current_region())

In [10]:
# create bucket if it not exists (private one)

In [11]:
default_bucket_name()

'tuplex-leonhard'

In [12]:
def ensure_s3_bucket(s3_client, bucket_name, region):
    bucket_names = list(map(lambda b: b['Name'], s3_client.list_buckets()['Buckets']))
    
    if bucket_name not in bucket_names:
        logging.info('Bucket {} not found, creating (private bucket) in {} ...'.format(bucket_name, region))
        
        # bug in boto3: 
        if region == current_region():
            s3_client.create_bucket(Bucket=bucket_name)
            logging.info('Bucket {} created in {}'.format(bucket_name, region))
        else:
            location = {'LocationConstraint': region.strip()}
            s3_client.create_bucket(Bucket=bucket_name,
                                    CreateBucketConfiguration=location)
            logging.info('Bucket {} created in {}'.format(bucket_name, region))
    else:
        logging.info('Found bucket {}'.format(bucket_name))
    
ensure_s3_bucket(s3_client, default_bucket_name(), current_region())

2021-11-04 12:37:07 INFO     Found bucket tuplex-leonhard


### Creating/uploading actual lambda function

In [13]:
lambda_client = boto3.client('lambda')

In [14]:
lambda_function_name=default_lambda_name()
lambda_zip_file = './tplxlam.zip'

try:
    response = lambda_client.get_function(FunctionName=lambda_function_name)
    print(response)
except lambda_client.exceptions.ResourceNotFoundException as e:
    logging.info('Function {} was not found in {}, uploading ...'.format(lambda_function_name, region))
    

2021-11-04 12:37:07 INFO     Function tuplex-lambda-runner was not found in us-east-1, uploading ...


In [ ]:
# from utils.common
try:
    import pwd
except ImportError:
    import getpass
    pwd = None
    
import datetime
import socket


import os
import sys
import threading

def current_user():
    """
    retrieve current user name
    Returns: username as string

    """
    if pwd:
        return pwd.getpwuid(os.geteuid()).pw_name
    else:
        return getpass.getuser()

def host_name():
    """
    retrieve host name to identify machine
    Returns: some hostname as string

    """
    if socket.gethostname().find('.') >= 0:
        return socket.gethostname()
    else:
        return socket.gethostbyaddr(socket.gethostname())[0]


def sizeof_fmt(num, suffix="B"):
    # from https://stackoverflow.com/questions/1094841/get-human-readable-version-of-file-size
    for unit in ["", "Ki", "Mi", "Gi", "Ti", "Pi", "Ei", "Zi"]:
        if abs(num) < 1024.0:
            return f"{num:3.1f}{unit}{suffix}"
        num /= 1024.0
    return f"{num:.1f}Yi{suffix}"

class ProgressPercentage(object):

    def __init__(self, filename):
        self._filename = filename
        self._size = float(os.path.getsize(filename))
        self._seen_so_far = 0
        self._lock = threading.Lock()

    def __call__(self, bytes_amount):
        # To simplify, assume this is hooked up to a single filename
        with self._lock:
            self._seen_so_far += bytes_amount
            percentage = (self._seen_so_far / self._size) * 100
            sys.stdout.write(
                "\r%s  %s / %s  (%.2f%%)" % (
                    self._filename, sizeof_fmt(self._seen_so_far), sizeof_fmt(self._size),
                    percentage))
            sys.stdout.flush()

def s3_split_uri(uri):
    assert '/' in uri, 'at least one / is required!'
    uri = uri.replace('s3://', '')
        
    bucket = uri[:uri.find('/')]
    key = uri[uri.find('/')+1:]
    return bucket, key



def upload_lambda(iam_client, lambda_client, lambda_function_name, lambda_role,
                  lambda_zip_file, overwrite=False, s3_client=None, s3_scratch_space=None):
    # AWS only allows 50MB to be uploaded directly via request. Else, requires S3 upload.
    
    ZIP_UPLOAD_LIMIT_SIZE=50000000 
    
    # Lambda defaults, be careful what to set here!
    # for runtime, choose https://docs.aws.amazon.com/lambda/latest/dg/lambda-runtimes.html
    RUNTIME="provided.al2"
    HANDLER="tplxlam" # this is how the executable is called...
    ARCHITECTURES=['x86_64']
    DEFAULT_MEMORY_SIZE=1536
    DEFAULT_TIMEOUT=30 # 30s timeout
    
    if not os.path.isfile(lambda_zip_file):
        raise Exception('Could not find local lambda zip file {}'.format(lambda_zip_file))
    file_size = os.stat(lambda_zip_file).st_size
    
    # if file size is smaller than limit, check how large the base64 encoded version is...
    CODE = None
    if file_size < ZIP_UPLOAD_LIMIT_SIZE:
        logging.info('Encoding Lambda as base64 ({})'.format(sizeof_fmt(file_size)))
        with open(lambda_zip_file, 'rb') as fp:
            CODE = fp.read()
            CODE = base64.b64encode(CODE)
            b64_file_size = len(CODE) + 1
            logging.info('File size as base64 is {}'.format(sizeof_fmt(b64_file_size)))
    else:
        b64_file_size = ZIP_UPLOAD_LIMIT_SIZE + 42 # to not trigger below if
        
    # get ARN of lambda role
    response = iam_client.get_role(RoleName=lambda_role)
    lambda_role_arn = response['Role']['Arn']
        
        
    # check if Lambda function already exists, if overwrite delete!
    l_response = lambda_client.list_functions(FunctionVersion='ALL')
    functions = list(filter(lambda f: f['FunctionName'] == lambda_function_name, l_response['Functions']))
    if len(functions) > 0:
        if len(functions) != 1:
            logging.warning('Found multiple functions with name {}, deleting them all.'.format(lambda_function_name))
        
        if not overwrite:
            raise Exception('Found existing Lambda function {}, specify overwrite=True to replace'.format(lambda_function_name))
        
        for f in functions:
            lambda_client.delete_function(FunctionName=f['FunctionName'])
            logging.info('Removed existing function {} (Runtime={}, MemorySize={}) from {}'.format(f['FunctionName'],
                                                                                         f['Runtime'],
                                                                                         f['MemorySize'],
                                                                                        f['LastModified']))
        
    logging.info('Assigning role {} to runner'.format(lambda_role_arn))
        
    user = current_user()
    host = host_name()

    DEPLOY_MESSAGE="Auto-deployed Tuplex Lambda Runner function." \
    " Uploaded by {} from {} on {}".format(user, host, datetime.datetime.now())
    
    
    if b64_file_size < ZIP_UPLOAD_LIMIT_SIZE:
        logging.info('Found packaged lambda ({})'.format(sizeof_fmt(file_size)))
        
        logging.info('Loading local zipped lambda...')

        logging.info('Uploading Lambda to AWS ({})'.format(sizeof_fmt(file_size)))
        try:
            # upload directly, we use Custom 
            response = lambda_client.create_function(FunctionName=lambda_function_name,
                                         Runtime=RUNTIME,
                                         Handler=HANDLER,
                                         Role=lambda_role_arn,
                                         Code={'ZipFile': CODE}, 
                                         Description=DEPLOY_MESSAGE,
                                         PackageType='Zip',
                                         MemorySize=DEFAULT_MEMORY_SIZE,
                                         Timeout=DEFAULT_TIMEOUT)
        except Exception as e:
            logging.error('Failed with: {}'.format(type(e)))
            logging.error('Details: {}'.format(str(e)[:2048]))
            raise e
    else:
        if s3_client is None or s3_scratch_space is None:
            raise Exception("Local packaged lambda to large to upload directly, " \
                            "need S3. Please specify S3 client + scratch space")
        logging.info("Lambda function is larger than current limit ({}) AWS allows, " \
                      " deploying via S3...".format(sizeof_fmt(ZIP_UPLOAD_LIMIT_SIZE)))
        
        # upload to s3 temporarily
        s3_bucket, s3_key = s3_split_uri(s3_scratch_space)
        
        # scratch space, so naming doesn't matter
        TEMP_NAME = 'lambda-deploy.zip'
        s3_key_obj = s3_key + '/' + TEMP_NAME
        s3_target_uri = 's3://' + s3_bucket + '/' + s3_key + '/' + TEMP_NAME
        s3_client.upload_file(lambda_zip_file, s3_bucket, s3_key_obj, Callback=ProgressPercentage(lambda_zip_file))
        logging.info('Deploying Lambda from S3 ({})'.format(s3_target_uri))
        
        try:
            # upload directly, we use Custom 
            response = lambda_client.create_function(FunctionName=lambda_function_name,
                                         Runtime=RUNTIME,
                                         Handler=HANDLER,
                                         Role=lambda_role_arn,
                                         Code={'S3Bucket': s3_bucket, 'S3Key' : s3_key_obj}, 
                                         Description=DEPLOY_MESSAGE,
                                         PackageType='Zip',
                                         MemorySize=DEFAULT_MEMORY_SIZE,
                                         Timeout=DEFAULT_TIMEOUT)
        except Exception as e:
            logging.error('Failed with: {}'.format(type(e)))
            logging.error('Details: {}'.format(str(e)[:2048]))
            
            # delete S3 file from scratch
            s3_client.delete_object(Bucket=s3_bucket, Key=s3_key_obj)
            logging.info('Removed {} from S3'.format(s3_target_uri))
            
            raise e
        
        # delete S3 file from scratch
        s3_client.delete_object(Bucket=s3_bucket, Key=s3_key_obj)
        logging.info('Removed {} from S3'.format(s3_target_uri))
    
    # print out deployment details
    logging.info('Lambda function {} deployed (MemorySize={}MB, Timeout={}).'.format(response['FunctionName'],
                                                                                     response['MemorySize'],
                                                                                    response['Timeout']))
    
    # return lambda response
    return response
        
        
s3_scratch = default_bucket_name() + '/scratch'
upload_lambda(iam_client, lambda_client, lambda_function_name, lambda_role, lambda_zip_file, True, s3_client, s3_scratch)

2021-11-04 13:55:54 INFO     Encoding Lambda as base64 (43.9MiB)
2021-11-04 13:55:54 INFO     File size as base64 is 58.6MiB


In [75]:
l_response = lambda_client.list_functions()

l_response

{'ResponseMetadata': {'RequestId': '4934da7f-3c60-448b-9f3b-82646ffec61d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 04 Nov 2021 17:51:37 GMT',
   'content-type': 'application/json',
   'content-length': '6537',
   'connection': 'keep-alive',
   'x-amzn-requestid': '4934da7f-3c60-448b-9f3b-82646ffec61d'},
  'RetryAttempts': 0},
 'Functions': [{'FunctionName': 's3demo',
   'FunctionArn': 'arn:aws:lambda:us-east-1:587583095482:function:s3demo',
   'Runtime': 'provided',
   'Role': 'arn:aws:iam::587583095482:role/lambda-demo',
   'Handler': 's3demo',
   'CodeSize': 22041513,
   'Description': '',
   'Timeout': 15,
   'MemorySize': 256,
   'LastModified': '2019-06-20T18:11:06.992+0000',
   'CodeSha256': 'SeVXy3ZKbqLt8MF+iwh/SkU+zDfGjzCn275rurh0CLM=',
   'Version': '$LATEST',
   'VpcConfig': {'SubnetIds': [], 'SecurityGroupIds': [], 'VpcId': ''},
   'TracingConfig': {'Mode': 'PassThrough'},
   'RevisionId': '127d255e-c0cb-4074-abe2-1e84d55d39b6',
   'PackageType': 'Zip'},
  {

In [ ]:
# need to specify the 

In [ ]:
!aws s3 cp s3://tuplex-public/tplxlam.zip . --request-payer requester

In [ ]:
!aws s3 cp s3://tuplex-public/tplxlam.zip .

In [ ]:
# Note: S3 will give fatal error: An error occurred (403) when calling the HeadObject operation: Forbidden in case.

In [ ]:
response = iam_client.get_role(RoleName=lambda_role)
print(response)

In [ ]:
iam_client.list_role_policies(RoleName=lambda_role)

In [ ]:
iam_client.attach_role_policy(RoleName=lambda_role, PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole')

In [ ]:
help(iam_client.put_role_policy)

In [ ]:
remove_lambda_role(iam_client, 'tuplex-lambda-role')

In [ ]:
!cat /var/folders/l7/8zgzcszx7z5gk7kk92f6nc1c0000gn/T/tmp8qrc12_k

In [ ]:
help(iam_client.create_role)

In [ ]:
iam_client = boto3.client('iam')

In [ ]:
iam = boto3.resource('iam')
account_summary = iam.AccountSummary()

In [ ]:
account_summary.load()

In [ ]:
account_summary.get_available_subresources()

In [ ]:
account_summary.summary_map

In [ ]:
user = iam.CurrentUser()

In [ ]:
user.user_name

In [ ]:
user.user_id

In [ ]:
user.get_available_subresources()